# Entrega

### Imports

In [27]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump, load

In [3]:
from preprocessing import reemplazarNulls,reemplazarCategoricas,reemplazarFechas,regularizar,targetBooleano

[###] All Done                                              


In [4]:
from utilities import score2, score

### Test Holdout

In [7]:
df_feat = pd.read_csv("datasets/holdout_features.csv", low_memory=False)
df_targ = pd.read_csv("datasets/holdout_target.csv")

In [8]:
df_targ=targetBooleano(df_targ).llovieron_hamburguesas_al_dia_siguiente
reemplazarNulls(df_feat , inplace=True)
reemplazarCategoricas(df_feat , inplace=True)
reemplazarFechas(df_feat , inplace=True)
regularizar(df_feat , inplace=True)

,id,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,mm_lluvia_dia,nubosidad_tarde,presion_atmosferica_tarde,rafaga_viento_max_direccion,rafaga_viento_max_velocidad,velocidad_viendo_tarde
0,-0.114786,-0.311222,-0.773796,-2.018030,-1.372610,-0.832696,0.345041,0.916539,-1.384970,-0.220589,1.544505,0.834079
1,1.006900,-1.358982,-0.976624,-2.018030,1.085157,0.638263,-0.281713,0.916539,-2.080499,0.802215,0.221665,-0.642648
2,0.008030,-1.149430,-0.570969,-2.018030,-0.023248,1.058537,1.020008,0.916539,-0.064885,-0.016028,-0.219282,-0.869837
3,0.337019,-0.939878,-0.976624,0.653494,-0.360588,-0.044682,-0.281713,0.916539,-0.717831,1.620459,-0.219282,0.152513
4,1.129419,-1.149430,-1.179451,-2.018030,0.362285,0.165455,-0.281713,0.181553,0.559671,-1.038833,-0.219282,0.379701
...,...,...,...,...,...,...,...,...,...,...,...,...
11368,0.913813,-0.101670,-0.773796,-1.515466,0.217710,0.270523,-0.257607,0.916539,0.971310,-0.834272,-0.513247,-0.188270
11369,0.358833,-1.568534,0.645996,-2.018030,-0.264205,-0.517490,-0.281713,-1.655914,-0.121663,-0.220589,-0.660229,-1.097025
11370,-1.586621,-0.101670,-0.570969,0.045127,-1.468993,-1.515641,-0.281713,0.916539,-1.015914,0.393094,3.014328,3.333155
11371,-1.313430,1.574745,1.254478,-2.018030,1.085157,0.795865,-0.209395,0.916539,1.553283,1.211337,0.221665,-0.415459


### Modelos

In [10]:
arbol = load('models/Tree/arbol_r.sk')

In [11]:
knn = load('models/KNN/knn_auto.sk')

In [12]:
nb = load('models/NB/nb_reg.sk')

In [13]:
svm_lin = load('models/SVM/svm_lineal.sk')
svm_pol = load('models/SVM/svm_poly.sk')
svm_rad = load('models/SVM/svm_radial.sk')

In [14]:
nn = load('models/NN/mlp_cls_r.sk')

In [15]:
rand_f = load('models/Ensambles/forest_r.sk')

In [16]:
boost = load('models/Ensambles/boost_r.sk')

In [17]:
# vote = load('models/Ensambles/vote_svm.sk')

## Preprocesamientos

preprocesamiento | descripcion | funcion
:--:|:--:|:--:
 convertir target a booleano | Convierte los 'si' y 'no' por True y False | `targetBooleano`
 reemplazar nulls de features | Reemplaza los nulls de los features con un `simple imputer` | `reemplazarNulls`
 reemplazar categoricas de features | convierte las features categoricas en numericas | `reemplazarCategoricas`
 reemplazar fechas de features | convierte las features de fecha en numericas | `reemplazarFechas`
 regularizar features | Normaliza las features y elimina las menos significativas | `regularizar`

## Comparacion con Test Holdout

In [18]:
modelos = [
    ('arbol',arbol),
    ('knn',knn),
    ('naive bayes',nb),
    ('svm lineal',svm_lin),
    ('svm poly',svm_pol),
    ('svm radial',svm_rad),
    ('Red Neuronal',nn),
    ('Random Forest',rand_f),
    ('Boosting',boost),
]

In [19]:
df = pd.DataFrame(columns=["Modelo", "Preprocesamientos", "AUC-ROC", "Accuracy", "Precision", "Recall", "F1 score"])

In [26]:
for (name, model) in modelos:
    pred = model.predict(df_feat)
    prob = model.predict_proba(df_feat)
    
    roc_auc, acc, prec, rec, f1 = score2(df_targ, pred, prob[:,1])
    
    df = df.append(
        {
            "Modelo": name,
            "Preprocesamientos":  "Todos",
            "AUC-ROC":  roc_auc,
            "Accuracy":  acc,
            "Precision":  prec,
            "Recall":  rec,
            "F1 score":  f1,
        },
        ignore_index=True
    )
df

,Modelo,Preprocesamientos,AUC-ROC,Accuracy,Precision,Recall,F1 score
0,arbol,Todos,0.857910,0.841027,0.722659,0.469941,0.569524
1,knn,Todos,0.870998,0.838565,0.770404,0.396857,0.523859
2,nb,Todos,0.829405,0.826167,0.631603,0.535560,0.579630
3,svm_lin,Todos,0.857328,0.838301,0.743785,0.423183,0.539444
4,svm_pol,Todos,0.860728,0.838917,0.765846,0.403536,0.528564
5,svm_rad,Todos,0.856471,0.838741,0.774094,0.394499,0.522644
6,nn,Todos,0.872029,0.846742,0.715360,0.523379,0.604493
7,rand_f,Todos,0.873621,0.845511,0.760582,0.451866,0.566921
8,boost,Todos,0.883838,0.851490,0.737514,0.522200,0.611456


## Conclusion

(CONCLUSIÓN) Finalmente luego de poner las tablas TABLA 1 y TABLA 2, nos piden que lleguemos a una conclusión sobre qué modelo recomendamos y por qué y que lo comparemos con respecto al baseline que anteriormente implementamos. También quiere que agreguemos un pequeño análisis de qué modelo elegiríamos si se necesitase tener la menor cantidad de falsos positivos o si necesitan tener una lista de todos los días que potencialmente lloverán hamburguesas al día siguiente sin preocuparse demasiado si metemos en la misma días que realmente no llovieron hamburguesas al día siguiente.

**Modelo Recomendado:** Boosting, evaluando con el test-holdout es el que mejores metricas en todos los campos excepto  Precision

- qué modelo elegiríamos si se necesitase tener la menor cantidad de falsos positivos


- qué modelo elegiríamos si se necesitase tener la menor cantidad de falsos positivos


- si necesitan tener una lista de todos los días que potencialmente lloverán hamburguesas al día siguiente sin preocuparse demasiado si metemos en la misma días que realmente no llovieron hamburguesas al día siguiente

#### Comparacion con Base Line

## Predicciones